In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy import stats

# For data cleaning
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [52]:
df = pd.read_csv('GlobalWeatherRepository.csv')
df.head()


,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,Afghanistan,Kabul,34.52,69.18,Asia/Kabul,1.715849e+09,2024-05-16 13:15,26.6,79.8,Partly Cloudy,...,8.4,26.6,1.0,1.0,04:50 AM,06:50 PM,12:12 PM,01:11 AM,Waxing Gibbous,55.0
1,Albania,Tirana,41.33,19.82,Europe/Tirane,1.715849e+09,2024-05-16 10:45,19.0,66.2,Partly cloudy,...,1.1,2.0,1.0,1.0,05:21 AM,07:54 PM,12:58 PM,02:14 AM,Waxing Gibbous,55.0
2,Algeria,Algiers,36.76,3.05,Africa/Algiers,1.715849e+09,2024-05-16 09:45,23.0,73.4,Sunny,...,10.4,18.4,1.0,1.0,05:40 AM,07:50 PM,01:15 PM,02:14 AM,Waxing Gibbous,55.0
3,Andorra,Andorra La Vella,42.50,1.52,Europe/Andorra,1.715849e+09,2024-05-16 10:45,6.3,43.3,Light drizzle,...,0.7,0.9,1.0,1.0,06:31 AM,09:11 PM,02:12 PM,03:31 AM,Waxing Gibbous,55.0
4,Angola,Luanda,-8.84,13.23,Africa/Luanda,1.715849e+09,2024-05-16 09:45,26.0,78.8,Partly cloudy,...,183.4,262.3,5.0,10.0,06:12 AM,05:55 PM,01:17 PM,12:38 AM,Waxing Gibbous,55.0


## Data Cleaning & Preprocessing

In [53]:
# Check for missing values
missing_values = df.isnull().sum()
print(missing_values)


country                         0
location_name                   0
latitude                        0
longitude                       0
timezone                        0
last_updated_epoch              0
last_updated                    0
temperature_celsius             0
temperature_fahrenheit          0
condition_text                  0
wind_mph                        0
wind_kph                        0
wind_degree                     0
wind_direction                  0
pressure_mb                     0
pressure_in                     0
precip_mm                       0
precip_in                       0
humidity                        0
cloud                           0
feels_like_celsius              0
feels_like_fahrenheit           0
visibility_km                   0
visibility_miles                0
uv_index                        0
gust_mph                        0
gust_kph                        0
air_quality_Carbon_Monoxide     0
air_quality_Ozone               0
air_quality_Ni

Looks like there no null values in the dataset. But that doesn't mean there are no missing values. We need to check for missing values in multiple ways.

### Handling Missing Values



In [54]:
## Let's create a sample dataframe with different types of missing values

missing_indicators = {'N/A', 'NA', 'missing', 'unknown', '?', '', ' ', -999}

# Function to check missing values
def check_missing_values(df):
    missing_values = df.isin(missing_indicators)
    return missing_values.sum()

# Check for missing values in the sample dataframe
print(check_missing_values(df_missingValues))

A    4
B    3
dtype: int64


### Looks like there are some missing values in the sample dataFrame. We would now replace the missing values from numerical dataframe with a median value and categorical columns with the most frequent value.

In [55]:
df.replace(missing_indicators, np.nan, inplace=True)

# Identify numeric and categorical columns
numeric_cols = df.select_dtypes(include=['number']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Replacing the missing values in numeric columns with a median value 
df[numeric_cols] = SimpleImputer(strategy='median').fit_transform(df[numeric_cols])

# Replace missing values in categorical columns with the most frequent value.
df[categorical_cols] = SimpleImputer(strategy='most_frequent').fit_transform(df[categorical_cols])

In [56]:
df.to_csv('GlobalWeatherRepository.csv', index=False)

### Handling the Outliers
Outliers are basically data points that Significantly differ from rest of the data points, to make sure our data set doesn't have extreme values we will handle the outliers as necessary.

We can do so by using the IQR method. In this we divided the data points in quartiles with Q1 having values of 25th percentile and more and Q3 having values 75th percentile and less. We clip this middle 50% to ge the best possible dataset

In [57]:
def check_outlier(df,col):
    Q1 = df[col].quantile(0.25)  # First quartile (25th percentile)
    Q3 = df[col].quantile(0.75)  # Third quartile (75th percentile)

    IQR = Q3 - Q1  # Interquartile Range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower_bound,upper_bound)
    return df


In [58]:
for col in numeric_cols:
    df = check_outlier(df, col)

## Normalize the data

In [59]:
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [62]:
df.describe()

,latitude,longitude,last_updated_epoch,temperature_celsius,temperature_fahrenheit,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,...,gust_kph,air_quality_Carbon_Monoxide,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,moon_illumination
count,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,...,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04,6.099800e+04
mean,7.077995e-17,2.425826e-17,4.517802e-15,2.255174e-16,4.715363e-16,-1.060025e-16,1.155543e-16,1.258051e-16,6.333124e-15,-2.011484e-14,...,2.422914e-16,3.168426e-17,-1.565575e-16,-7.082363e-17,-1.258051e-16,2.236536e-17,-2.329725e-16,-1.192819e-16,-1.714677e-16,2.982048e-17
std,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,...,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00
min,-2.469254e+00,-1.901437e+00,-1.725594e+00,-2.441618e+00,-2.445272e+00,-1.174019e+00,-1.167624e+00,-1.621285e+00,-2.412059e+00,-2.304816e+00,...,-1.416735e+00,-2.602523e+00,-1.885998e+00,-7.831660e-01,-2.370214e+00,-1.048160e+00,-2.528608e+00,-8.001904e-01,-7.862367e-01,-1.431686e+00
25%,-6.286253e-01,-4.730638e-01,-8.573838e-01,-5.807972e-01,-5.830583e-01,-8.213099e-01,-8.144367e-01,-8.594388e-01,-6.139730e-01,-5.960722e-01,...,-7.471769e-01,-7.740708e-01,-7.151851e-01,-7.171395e-01,-7.233683e-01,-7.674448e-01,-7.594556e-01,-8.001904e-01,-7.862367e-01,-9.730187e-01
50%,-7.704955e-02,2.702187e-02,7.627646e-03,2.961412e-01,2.960729e-01,-2.530556e-01,-2.420292e-01,-8.794907e-02,-1.644516e-01,-1.300513e-01,...,-2.078108e-01,-3.394821e-01,-6.073062e-02,-5.333362e-01,-5.134762e-01,-3.595221e-01,-3.784758e-01,-8.001904e-01,-2.089546e-01,1.648624e-03
75%,8.688007e-01,4.791850e-01,8.638416e-01,6.597499e-01,6.584176e-01,6.287183e-01,6.348504e-01,8.378385e-01,5.847507e-01,5.430901e-01,...,6.105378e-01,4.448976e-01,6.057322e-01,4.053099e-01,3.745291e-01,4.583466e-01,4.199792e-01,3.897173e-01,3.683275e-01,9.763160e-01
max,1.839165e+00,1.907558e+00,1.718799e+00,2.520571e+00,2.520631e+00,2.803761e+00,2.808781e+00,1.840775e+00,2.382836e+00,2.251834e+00,...,2.647110e+00,2.273350e+00,2.587108e+00,2.088984e+00,2.021375e+00,2.297034e+00,2.189131e+00,2.174579e+00,2.100174e+00,1.434983e+00


Looks like we have successfully completed our data cleaning process. Let's save our updated csv file.

In [63]:
df.to_csv('GlobalWeatherRepository_cleaned.csv', index=False)

# Exploratory Data Analysis (EDA)